The Clemency Initiative
----

In our last meeting we started looking at [the web page of Obama's commutations](https://www.justice.gov/pardon/obama-commutations). Our ultimate goal was to create a data frame that would let us operate on the data more conveniently. So how do we take the free text of [the commutations web page](https://www.justice.gov/pardon/obama-commutations) and systematically fill in a more structured data set? 

**1. Cleaning up the "item"-"description" pairs**

Let's start where you lesson left off...

In [ ]:
library(rvest)
library(dplyr)
library(lubridate)
library(reshape2)
library(stringr)

page = read_html("https://www.justice.gov/pardon/obama-commutations")
tbs = html_nodes(page,"table")

t1 = html_table(tbs[[1]])
names(t1) = c("item","description")

head(t1)

While there are probably much more elegant ways to do this, we are going to run over the nodes in the web page that are tables, and then add them to a growing data frame. Each pass of the loop adds the data from another table. 

In [ ]:
# start with t1 and then loop through the other tables, adding to
# the big "commutations" data set

commutations = t1

for(i in 2:length(tbs)){
    
    tt = html_table(tbs[[i]])
    
    # Table 9 has 3 columns by mistake - the third is unnecessary 
    # so we take just the first two in all cases
    
    tt = tt[,1:2]
    
    # Rename the columns from X1 and X2
    
    names(tt) = c("item","description")
    
    # Add the current table to the end of "commutations" by row-binding them
    
    commutations = rbind(commutations,tt)
}

In [ ]:
dim(commutations)

This gives us 8750 lines or so. Now, let's look at those that have a blank "item" field. We'll notice that some of them are names, some are second or third elements of the offense or sentence, and some are paired with a second blank field. We'll want to get rid of those.

In [ ]:
sample_n(filter(commutations,str_trim(item)==""),25)

This leads me to ask whether any other "description" and "item" fields are the same.

In [ ]:
filter(commutations,str_trim(item) == str_trim(description))

So let's get rid of these. They are going to mess things up.

In [ ]:
commutations = filter(commutations,str_trim(item)!="" | str_trim(description)!="")
commutations = filter(commutations,str_trim(item) != str_trim(description))

In [ ]:
head(commutations,50)

In [ ]:
table(commutations$item)

Now, we want to fill in the "item" field and replace the blank lines with something else. There are patterns that emerge in the data. Can you describe them?

There are certainly better ways to do this, but in the code below, we run over each row and look at the row above it. If the "item" field is blank, we will look up and see if it had "Terms of grant:" or some other phrase above it and fill in the blank field accordingly. What is the rule?

In [ ]:
itemm1 = commutations$item[1]

for(i in 2:nrow(commutations)){
    
    item = commutations$item[i]
    
    if(str_trim(item)==""){
    
        if(tolower(str_trim(itemm1))=="terms of grant:"){
            commutations$item[i] = "Name:"
        }
        else{
            if(tolower(str_trim(itemm1))=="district/date:"){
                commutations$item[i] = "District/Date 2:"
            }
            else{
                if(tolower(str_trim(itemm1))=="sentence:"){
                    commutations$item[i] = "Sentence 2:"
                }
                else{
                    if(tolower(str_trim(itemm1))=="offense:"){
                        commutations$item[i] = "Offense 2:"
                    }
                }
            }
        }

    }

    itemm1 = commutations$item[i]
}    

commutations[1,1] = "Name:"

In [ ]:
head(commutations,50)

In [ ]:
dim(commutations)

In [ ]:
table(commutations$item)

Let's look at the "item" cells that are blank...

In [ ]:
commutations[str_trim(commutations$item)=="",]

These we fix manually.

In [ ]:
commutations[4315:4325,]

In [ ]:
commutations$item[4321] = "District/Date:"
commutations$description[4321] = NA
commutations$item[4322] = "District/Date 2:"

In [ ]:
commutations[4315:4325,]

OK one fixed. Now let's tackle the next one.

In [ ]:
commutations[str_trim(commutations$item)=="",]

In [ ]:
commutations[4415:4425,]

Checking the web page, we see that there was one offense listed. We might need to do more checking here, but for now we'll assume it's just one offense and change the data.

In [ ]:
commutations = commutations[-4418,]
commutations[4418,]

In [ ]:
commutations$item[4418] = "District/Date:"
commutations$description[4418] = "District of New Mexico; August 7, 2006"
rownames(commutations) = 1:nrow(commutations)

And check.

In [ ]:
commutations[4415:4425,]

In [ ]:
commutations$description[4417] = "Possession with intent to distribute 500 grams and more of a mixture"
commutations$description[4419] = "360 months’ imprisonment; 10 years’ supervised release"
commutations[4415:4425,]

Two fixed. Now the third.

In [ ]:
commutations[str_trim(commutations$item)=="",]

In [ ]:
commutations[6240:6250,]

In [ ]:
commutations$item[6244] = "Offense 3:"
commutations$item[6247] = "District/Date 3:"
commutations$item[6250] = "Sentence 3:"
commutations[6240:6250,]

Ok, onto the next one.

In [ ]:
commutations[str_trim(commutations$item)=="",]

Fixed! Ha! We now go thorugh and fix the "item" entries. We will get rid of slashes and spaces and remove any whitespace. We will also turn the labels all lowercase. 

In [ ]:
table(commutations$item)

In [ ]:
commutations$item = tolower(str_trim(commutations$item))
commutations$item = gsub("/","_",gsub(":","",gsub(" ","_",commutations$item)))
commutations$description = gsub("’","'",commutations$description)

In [ ]:
table(commutations$item)

Finally, we want to give everyone an ID. We'll create a column made of 0's and then put a 1 where we have a "name" field. We will then take the cumsum() of this column to give everyone a unique ID.

In [ ]:
commutations$id = 0
commutations$id[commutations$item=="name"] = 1
commutations$id = cumsum(commutations$id)

In [ ]:
head(commutations,25)

In [ ]:
table(table(commutations$id))

The 4 and the 11 look suspicious. Let's check those out.

In [ ]:
tt = table(commutations$id)
tt[tt==4]

In [ ]:
commutations[commutations$id==876,]

For ID 876, we are missing a "district_date" entry. We can see the district in the offense entry. Let's add it and fix the offense entry.

In [ ]:
commutations$description[4460] = "Conspiracy to distribute in excess of 500 grams of methamphetamine (mixture)"
tmp = data.frame(item="district_date",description="Southern District of Indiana",id=876)
commutations = rbind(commutations,tmp)

Now check the 11 entry id.

In [ ]:
tt[tt==11]

In [ ]:
commutations[commutations$id==1225,]

The 11 is OK because it corresponds to (our only) an entry with 3 offenses and sentences and districts. Look at our table of counts again and see if there is anything else to fix up.

In [ ]:
table(table(commutations$id))

In [ ]:
tt[tt==8]

In [ ]:
commutations[commutations$id==1166,]

Let's now look at the few that are potentially malformed. These have just 6 or 7 entries.

In [ ]:
malformed = as.numeric(names(tt[tt==6 | tt==7]))
malformed

... and for the moment we will remove them. 

In [ ]:
commutations = filter(commutations,!(id %in% malformed))
table(table(commutations$id))

Finally check that our categories all line up.

In [ ]:
(table(commutations$item))

**2. Transforming the shape to a regular data frame**

Now given this shape of the data set, we can fold it back up using dcast(). The interplay between the two formats is really common.

In [ ]:
library(reshape2)
newcommutations = dcast(commutations,id~item,value.var = "description")
head(newcommutations)

**3. Adding new variables to the data set**

Make dates as we did before, knowing we'll have to fix some of them up manually. 

In [ ]:
newcommutations = mutate(newcommutations,date=mdy(str_extract(district_date,"; .*")))

In [ ]:
newcommutations = filter(newcommutations,!is.na(date))

In [ ]:
newcommutations = mutate(newcommutations,date_2=mdy(str_extract(district_date_2,"; .*")))
newcommutations = mutate(newcommutations,date_3=mdy(str_extract(district_date_3,"; .*")))

head(newcommutations)

Now, let's pull the states from the "district" field. R has a data set referring to names of states. We load it by using a call to data() -- similar to the library() call.

In [ ]:
data(state)
ls()

In [ ]:
state.name

Let's make a regular expression now by pasting these names together with a giant "or". We are looking for "\bname\b" in each case.

In [ ]:
reg = str_c("\\b",tolower(state.name),"\\b",collapse="|")
reg

In [ ]:
dd = newcommutations$district_date
dd[is.na(str_extract(tolower(dd),reg))]

In [ ]:
newcommutations$district_date[126]="Eastern District of Wisconsin; March 13, 2007"

In [ ]:
state.name = c(state.name,"Puerto Rico","Guam","District of Columbia","U.S. Army Court Martial")
reg = str_c("\\b",tolower(state.name),"\\b",collapse="|")

dd = newcommutations$district_date
dd[is.na(str_extract(tolower(dd),reg))]

In [ ]:
newcommutations = mutate(newcommutations,state=str_extract(tolower(district_date),reg))

And make a table of the states represented.

In [ ]:
newcommutations = mutate(newcommutations,state_2=str_extract(tolower(district_date_2),reg))
newcommutations = mutate(newcommutations,state_3=str_extract(tolower(district_date_3),reg))

head(newcommutations)

In [ ]:
table(newcommutations$state)

In [ ]:
sort(table(newcommutations$state))

Now the sentences. First, have a look. Let's try to pull the count of months and years.

In [ ]:
head(newcommutations$sentence)

In [ ]:
tmp = str_extract(newcommutations$sentence,"[0-9]+ [^0-9]+ imprisonment")
tmp

Lots missing. Gotta work harder. Let's try something about life imprisonment or a count of months/years.

In [ ]:
print(newcommutations$sentence[is.na(tmp)])

In [ ]:
tmp = str_extract(tolower(newcommutations$sentence),"^(1\\.)?\\s*(240 months)|([0-9,]+ [^0-9]+ imprisonment)|(life imprisonment)|(life plus (five years'||[0-9]+ [^0-9]*) imprisonment)")
tmp

In [ ]:
term = "^(1\\.)?\\s*(240 months)|([0-9,]+ [^0-9]+ imprisonment)|(death sentence)|(life imprisonment)|(life plus (five years'||[0-9]+ [^0-9]*) imprisonment)"
newcommutations = mutate(newcommutations,term_string = str_extract(tolower(sentence),term))
head(newcommutations)

Finally, let's turn the months into years so we're all in the same units.

In [ ]:
newcommutations$time = as.numeric(str_extract(newcommutations$term_string,"^([0-9]+|life)"))/ifelse(str_detect(newcommutations$term_string,"month"),12,1)
sample_n(newcommutations,10)

In [ ]:
newcommutations$term_string[is.na(newcommutations$time)]

In [ ]:
hist(newcommutations$time,breaks=100)

In [ ]:
filter(newcommutations,time>80)

In [ ]:
sample_n(newcommutations,20)

The offense field looks good. Let's see if we can pull out the laws that were broken. I love the section symbol...

In [ ]:
dim(filter(newcommutations,!str_detect(offense,"§")))

In [ ]:
head(filter(newcommutations,!str_detect(offense,"§")))

It's missing in 201 of the 945 commutations. Not so good. Let's try to scrape out different drugs instead.

In [ ]:
nodrugs = filter(newcommutations,!(str_detect(tolower(offense),"phencyclidine|narcotic|drug|controlled substance|heroin|cocaine|mari(j|h)uana|methamphetamine")))
nodrugs

In [ ]:
dim(nodrugs)

Finally, we'll venture into Python with the first names of our inmates and try to assess their gender.

In [ ]:
newcommutations = mutate(newcommutations,first_name=str_extract(name,"^[a-zA-Z]+"))

Include just the first name...

In [ ]:
newcommutations = newcommutations[,c(1,5,21,2:4,13:18,6:11,19:20,12)]

... and write it out to a file. We will then use a Python program to add gender (slightly iffy, but we can try).

In [ ]:
write.csv(newcommutations,file="newcomms.csv",row.names=FALSE)

Either read it back in or download the newcomms2.csv file from CourseWorks.

In [ ]:
newcommutations = read.csv("newcomms2.csv")

dim(newcommutations)

In [ ]:
head(newcommutations)

In [ ]:
table(newcommutations$gender)

In [ ]:
sample_n(filter(newcommutations,gender=="unknown"),25)

In [ ]:
library(lattice)

histogram(~time|gender,data=newcommutations,breaks=80)